In [ ]:
import pandas as pd
import os
import numpy as np

# pip install tensorflow Version: 2.17.0
import tensorflow as tf

from tensorflow.python.keras import models, layers

# Version: 3.4.1
from tensorflow import keras

# use for splitting the test and train data
from sklearn.model_selection import train_test_split

# from scipy import spatial

# import matplotlib.pyplot as plt

# for the creation of the cartesian product grid
from itertools import product

# for the timing of each test
import time

import math
import json
import random
from IPython.display import clear_output
from datetime import datetime

import pprint

# the data tools module
import data_tools as dt
dh = dt.Data_Handling(output_size=5)

# set the paths to used by physionet
label_path = 'mitdb_labels_reduced.npy'
data_path = 'mitdb_data_reduced.npy'

# get the data
dh.load_data(label_path=label_path, data_path=data_path)

# split into train and test sets
dh.split_data(split=0.2)

print(dh.X_train.shape, dh.X_test.shape)

In [13]:
class Population:
    '''
    Class to create a population of genomes with each containing genes of random float values
    
    attributes:
        layer_size
        layer_shape
    '''
    def __init__(self, max_layers=5, global_len=3, layer_len=3, dp=4):
        self.__max_layers = max_layers
        self.__global_len = global_len
        self.__layer_len = layer_len
        self.__layer_shape = None
        self.__dp = dp

#     @property
#     def global_size(self):
#         return self.__global_size
    
    @property
    def layer_size(self):
        return self.__layer_size
    
    @property
    def layer_shape(self):
        return self.__layer_shape
    
    @property
    def max_layers(self):
        return self.__max_layers
    
    def get_genome(self, layers=1):
        '''
        Creates a genome to include a global gene and at least 1 layer
        
        Params:
            size (int) the number of layers within the genome
            dp (int) the number of decimal places for each value in the genes
        Returns:
            (np.array) a set of genes
        '''
        assert layers >= 1, "Size must be at least 1"

        # golbal gene
#         genome = list(self.get_gene(size=self.__global_len))
        genome = self.get_gene(size=self.__global_len)
        # layers from random genes
        layers = [self.get_gene(size=self.__layer_len) for gene in range(layers)]
        genome.extend(layers)
        
        return genome
    
    
    def get_gene(self, size=4):
        '''
        Creates gene of the given size with a set number of decimal places
        
        Params:
            size (int) the number of values within the gene
            dp (int) the number of decimal places for each value in the gene
        
        Returns:
            (np.array) a single gene
        '''
        
#         set in powers of 10 to control the decimal places
        ubound = 10 ** self.__dp
#         create a random gene of the given size
        gene = np.random.randint(0, ubound,  size=size)/ubound
#         pad with -1s to max size to create uniform size

        return list(gene)
    
    def get_population(self, pop_size=3):
        '''
        Creates a population of the given size with random length genomes
        
        Params:
            pop_size (int) the number random genomes to return
        
        Returns:
            (np.array) a set of genomes
        '''
        population = []
        for i in range(pop_size):
            
            layers = np.random.randint(1, self.__max_layers + 1)
            population.append(self.get_genome(layers=layers))

        return population

pn = Population(max_layers=4, global_len=mc.global_len, layer_len=mc.layer_len, dp=1)   

In [18]:
pop = pn.get_population(3)
save_dict(pop)

In [3]:
class Model_Constructor:
    '''
    handles the stages of mapping a genome to the required set of 
    hyper-parameter values of various types. these are used to build and
    compile a training ready model

    parameters
        shape (tuple) specifies the input shape of the data to be modelled eg (28, 28, 1) 
        output_size (int) the number of classes to be modelled eg 5
        
    attributes:
        gene_spec: the specification for each genome
        layer_len: the length of the current layer config gene
        global_len: the length of the global parameter gene

    '''
    
    def __init__(self, shape, output_size):
        
        self.set_gene_spec()
#         self.__layer_len = None
#         self.__global_len = None
        self.__shape = shape
        self.__output_size = output_size
        
    def set_gene_spec(self):
        '''
        sets up the genespec for use with the model constructor 
        gene values are mapped to the ranges specifiec using the map_params function
        '''
        start_sizes = [i for i in range(8, 33, 8)]
        dropout_rates = [i/10 for i in range(2, 6, 1)]
        filters = [i/10 for i in range(11, 20)]
        binary = [True, False]
        batch_sizes = [v for v in range(64, 256, 32)]
        
        self.__gene_spec = {
            'global':{
                'learning_rate':[0.1, 0.01, 0.001, 0.0001],
                'optimizer':['adam', 'sgd'],
                'start_size':start_sizes,
                'batch_size':batch_sizes,
            },
            'layer':{
    
                    'filter_size':filters,
                    'filter_activation': ['relu'],
                    'dropout_exists': binary,
                    'dropout_rate': dropout_rates,
                    'max_pool_exists': binary,
                    'max_pool_size' : [2, 3]
            }   
        }
        # set the parameter lengths which in the case of the layers is a single layer length
        self.__layer_len = len(self.__gene_spec['layer'])
        self.__global_len = len(self.__gene_spec['global']) 
        
    @property
    def gene_spec(self):
        return self.__gene_spec
    
    @property
    def layer_len(self):
        return self.__layer_len
    
    @property
    def global_len(self):
        return self.__global_len
         
    def map_params(self, gene_spec, gene):
        '''
        maps a gene's values to the corresponding range in the gene_spec
        
        parameters:
            gene_spec (dict) a subsection of the genespec eg global or layer
            gene (array float) set of numbers used to map to the spec's ranges
        
        returns:
            (dict) the parameters mapped to the gene
        '''
        params = {}
        for i, key in enumerate(gene_spec.keys()):
            idx = math.floor(len(gene_spec[key]) * gene[i])
            params[key] = gene_spec[key][idx]

        return params
    
    def get_param_dict(self, genome):
        '''
        creates a dictionary containing the useable parameters for the model components
        
        parameters:
            genome (2d array float) containing genes to construct a model
        
        returns:
            (dict) the mapped global parameters
            (dict) the mapped local parameters
        
        '''
        # split the golbals and map to dict
        spec = self.__gene_spec['global'].copy()
        gene = genome[:self.__global_len]
        global_params = self.map_params(spec, gene)

        # split out the layer genes and loop over to map
        layer_params = {}
        spec = self.__gene_spec['layer'].copy()
        gnm = genome[self.__global_len:]

        for i, gene in enumerate(gnm):
            layer_params[f'layer_{i}'] = self.map_params(spec, gene)
            
        return global_params, layer_params
    
    def build_model(self, global_params, layer_params):
        '''
        builds a keras model of varying depth.
        the depth is defined by the length of the layer_params
        layers such as dropout and pooling are added if the parameters specifies true
        each layer will have at least 1 conv2d and at most conv2d, maxpool, dropout
        parameters are defined by the two sets passed in
        
        parameters:
            global_params (dict) parameters common to the model as a whole
            layer_params (dict) parameters specific to the hidden layers
            
        returns:
            (keras.model) a compliled model ready for training
        
        '''
        # used for the scaling of each filter size
        prev_size = global_params['start_size']
        # new empty model
        model = keras.Sequential()
    #     # set the input shape
        model.add(keras.Input(shape=self.__shape))
        
        for i, key in enumerate(layer_params.keys()):
            # extract the current layer's parameters to make code more readable
            params = layer_params[key]
            # calculate the layer size base on the previous size
            layer_size = int(params['filter_size'] * prev_size)
            
            # CONV2D
            model.add(
                keras.layers.Conv2D(
                    filters=layer_size, 
                    kernel_size=3, 
                    activation=params['filter_activation']
                )
            )
            # MAX POOL
            model.add(
                keras.layers.MaxPooling2D(
                    pool_size=params['max_pool_size']
                )
            )
            # DROPOUT
            if params['dropout_exists']:
                # only add a droput layer if param = true
                model.add(
                    keras.layers.Dropout(
                        rate=params['dropout_rate']
                    )
                )

            prev_size = layer_size
            
        # OUTPUT
        model.add(keras.layers.Flatten())
        model.add(keras.layers.Dense(self.__output_size, activation="softmax"))      

        # compile the model
        model.compile(optimizer=global_params['optimizer'],
                      loss="sparse_categorical_crossentropy",
                      metrics=["accuracy"])

        return model

In [15]:
# new model constructor to handle the build and training of each genome
mc = Model_Constructor(shape=dh.shape, output_size=5)
# new population constructor to create sets of genomes for each successive generation
pn = Population(max_layers=4, global_len=mc.global_len, layer_len=mc.layer_len, dp=1)

### For the report

In [ ]:
pop = pn.get_population(pop_size=5)
genome = pop[0]
pprint.pp(genome)

In [ ]:
global_params, layer_params = mc.get_param_dict(genome)
pprint.pp(global_params)
pprint.pp(layer_params)

In [ ]:
model = mc.build_model(global_params, layer_params)
model.summary()

In [ ]:
class GA_Optimizer:
    
    def __init__(self, X, y):
        self.__X = X
        self.__y = y
        
    def test_population(self):
        pass
    
    def evolve_the_model(self):
        pass

In [5]:
def save_dict(dict):

    file_name = f'{str(datetime.now())[:16]}_ga_results.json'
    f = open(file_name, "w")

    json.dump(dict, f, indent = 6)

    f.close()
    

In [16]:
def test_population(population, X, y, max_epochs, validation_split=0.2, callbacks=[], monitor_string=""):
    
    best_results = []
    best_epochs = []
    best_so_far = float("inf")
    for i, genome in enumerate(population):
        # map the gene and build the model
        g, l = mc.get_param_dict(genome)
        model = mc.build_model(g, l)
        # monitor metrics 
        depth = len(l.keys())
        batch_size = g['batch_size']
        
        # process update to user
        clear_output()
        print(f'{monitor_string}')
        print(f'Testing run: \t| {i + 1}/{len(population)}')
        print(f'Depth:\t\t| {depth}')
        print(f'Batch size:\t| {batch_size}\n')
        print(f'Best this gen: {best_so_far}\n')
        
        # fit to the data 
        model.fit( 
                    X, 
                    y,
                    epochs=max_epochs, 
                    validation_split=validation_split, 
                    batch_size=g['batch_size'], 
                    callbacks=[callbacks],
                    verbose=1
                )
        
        # get the training history
        hist = model.history.history['val_loss']
    #   store best fitness and the best epoch for survival of the fittest
        best_result = hist[-1]
        # for monitoring during the test
        best_so_far = min(best_so_far, best_result)
        # for output for use in the Evolution class
        best_epochs.append(len(hist))
        best_results.append(best_result)
        
        
    return best_results, best_epochs
    

def evolve_the_model(
    X, y, generations, pop_size, start_epochs, epoch_factor, fitness_func):
    
    # create the callbacks
    monitor = 'val_loss'

    callbacks = [keras.callbacks.EarlyStopping(
            monitor=monitor, 
            patience=3,
            mode='min'
    )]
    results = {}
    file_path = 'ga_results.json'
    
    epochs=start_epochs
    # for monitoring
    best_so_far = float("inf")
    # get initial population of size n
    pop = pn.get_population(pop_size=pop_size)
    # evolve for the number of generations
    for generation in range(generations):
        
        results[generation] = {'population': pop, 'epochs':epochs}
#         save_dict(results)
        
        monitor_string = f'Generation\t| {generation + 1} of {generations} for {epochs} epochs\n'
        monitor_string += f'Best so far:\t| {best_so_far}\n'
        
        fits, best_epochs = test_population(
            population=pop, 
            X=X, 
            y=y, 
            max_epochs=epochs, 
            validation_split=0.2, 
            callbacks=callbacks, 
            monitor_string=monitor_string)
        
        
        fittest = min(fits)
        best_so_far = min(best_so_far, fittest)
        
        results[generation].update({'fits':fits, 'best_epochs':best_epochs})
#         save_dict(results)
        
        epochs = epochs + epoch_factor
    # get next pop from the Evolution class of size n
    # EDIT THIS TO THE CORRECT POP SOURCE!
#         print(f'Next generation')
        pop = fitness_func(fits, pop)
        
    return results
    # can also reduce the population using the same idea - try as an experiment
    
    # also try using the early stopping as a bespoke in that if performance has
    # not improved for n generations then end
    

In [17]:
X = dh.X_train
y = dh.y_train

results = evolve_the_model(
    X=X, y=y, 
    generations=10, 
    pop_size=20, 
    start_epochs=10, 
    epoch_factor=2, 
    fitness_func=ev.get_next_generation
)

Generation	| 1 of 10 for 10 epochs
Best so far:	| inf

Testing run: 	| 1/20
Depth:		| 3
Batch size:	| 160

Best this gen: inf

Epoch 1/10
46/57 ━━━━━━━━━━━━━━━━━━━━ 7s 704ms/step - accuracy: 0.4976 - loss: 16.5573

KeyboardInterrupt: 

In [ ]:
results

In [ ]:
results = {}
results[1] = {'population':list(pop[0][4])}
save_dict(results)

results


In [ ]:
pop = pn.get_population(pop_size=5)

pop

In [ ]:
best_results, best_epochs = test_population(
    population=pop, X=X, y=y, max_epochs=2, validation_split=0.2, callbacks=[], monitor_string="Test"
)
print(best_results)

In [ ]:
# see twitter bookmark
from alive_progress import alive_bar

with alive_bar() as bar:

In [ ]:
# rebuild and train on entire set
model = mc.build_model(g, l)
vaidation_split = 0

mc.train_model(
    model, 
    X, 
    y, 
    epochs=max_epochs, 
    split=vaidation_split, 
    batch=g['batch_size']
)

In [ ]:
model.evaluate(X, y, batch_size=g['batch_size'])

In [ ]:
fits

### Repeatability
- The default initializer is random_glorot with a default seed=None.   
- This, according to keras, produces a deterministic set of values https://keras.io/api/layers/initializers/  
- "Note that an initializer seeded with an integer or None (unseeded) will produce the same random values across multiple calls"

In [8]:
class Evolution:
    '''
    Requires the Genome class using the static functions - instance not required
    
    Params:
            probability (float) the rate of probability eg 1 is 100% and 0.5 is 50%
            highest_is_fittest (bool) if true then higher values are considered fitter (eg accuracy)
                          if false thenlower values are considered fitter (eg loss)
            aggression (float) a higher number gives more weight to the fitter values giving a higher
                                probability of these being chosen in the selection function
            global_len (int) the length of the global gene in which each paramter represented by a single value
            mutation_amount (float [0.0, 0.1]) the amount of mutation to be applied to any gene value 
    '''
    
    def __init__(
        self, 
        probability=0.1,
        highest_is_fittest=True, 
        aggression=2,
        global_len=4,  
        mutation_amount=0.01,
        max_depth=5
    ):
        self.__probability = probability
        self.__highest_is_fittest = highest_is_fittest 
        self.__aggression = aggression
        self.__global_len = global_len 
        self.__mutation_amount = mutation_amount
        self.__max_depth = max_depth

    def probability_test(self):
        '''
        Returns true if the random number is below that of the rate parameter
        This gives a probabilty test for any defined operations
        
        Returns:
            (bool) true if the random number is less than the rate parameter
        '''
        return True if np.random.rand() < self.__probability else False

    def select_fittest(self, fits):
        '''
        selects the fittest value from a list using the roulette method and returns 
        the index value of the original source list which can then be applied to a 
        list of genes or parameters if build using the same order
        
        parameters:
            fits (array float) fitness values for a set of models

        returns:
            (int) the index value of the fittest value in the list
        '''
        # arbitary figure to ensure all significant values are integers
        int_scale = 10000
        
        if self.__highest_is_fittest:
            fits = (np.array(fits) * int_scale).astype(int)
        else:
#           invert the values in order to prioritize the lowest
            fits = (1 /np.array(fits) * int_scale).astype(int)
        # order the values such that the larger have a wider interval
        cum_array = (np.cumsum(np.sort(fits)) * self.__aggression).astype(int)
        # choose random int between zero and max of the cum array scaled by the exponent 1/aggression
        random_idx = int((np.random.rand() ** (1/self.__aggression)) * cum_array[-1])
#         find the corresponding index value within the cum array
        idx = np.searchsorted(cum_array, random_idx, side="left")

        # Get the sorted indices of the array
        sorted_indices = np.argsort(fits)
    #     extrapolate back the index to that of the corresponding value in the fits array
        res = sorted_indices[idx]

        return res
    
    def point_mutate(self, genome):
        '''
        Randomly mutates a gene or genes within the given set of genes. 
        The gene values are either increased or decreased depending on the random number -0.5 to 0.5. 

        Params:
            genome (np.array) the set of genes to be mutated

        Returns:
            (list) the mutated set of genes
        '''

        for i, element in enumerate(genome):
            # only layer genes will have the __len__ attribute
            if hasattr(element, "__len__"):
                # sub divide any genes into individual scalars and mutate
                for j, sub_element in enumerate(element):
                    genome[i][j] = self.mutate_gene_value(sub_element)
            else:
            # others will be scalar global gene values
                genome[i] = self.mutate_gene_value(element)
            
        return genome
    
    def mutate_gene_value(self, gene_value):
        
        if self.probability_test():
            
            mutation = (np.random.rand() - 0.5) * self.__mutation_amount
            gene_value = gene_value + mutation
            # fix value in the interval [0.0, 1.0] 
            gene_value = min(max(gene_value, 0), 1)
            
        return gene_value
    
    def shrink_mutate(self, genome):
        '''
        Probabilistically removes one gene to the genes set resulting in one less layer
        
        Params:
            genome (np.array) set of genes to be mutated
            
        Returns:
            (np.array) set of genes with len genes or genes - 1
        '''
        min_depth = 2
#         ensure that the deep layer count is always >= 1

        depth = self.get_genome_depth(genome)

        if depth <= min_depth: return genome

        if self.probability_test():
            idx = np.random.randint(1, depth)
            genome = genome[:-idx]

        return genome
    
    def grow_mutate(self, genome):
        '''
        Probabilistically adds one gene to the genes set, resulting in another layer
        
        Params:
            genome (np.array) set of genes to be mutated
            
        Returns:
            (np.array) set of genes with len genes or genes + 1
        '''
        new_genome = []
        layer_width = len(genome[-1])
        depth = self.get_genome_depth(genome)
        
#       ensure layers never exceeds the maximum number of layers
#       split at the global params index to get layers len
        if depth >= self.__max_depth:
               return genome

        if self.probability_test():
        # adds a new layer but not exceeding the max layers
            new_gene = pn.get_gene(size=layer_width)
            genome = genome + list([new_gene])
                
        return genome
    
    def crossover(self, genome_1, genome_2):
        '''
        Merges two genes at a random point to create a new "child" gene
        
        Params:
            gene1 (np.array) single gene
            gene2 (np.array) single gene
            
        Returns:
            (np.array) set of genes with len genes or genes + 1
        '''
#         use min() to ensure the index remains in the bounds 
#         of the smaller gene array
        idx = random.randint(
            0, min(len(genome_1), len(genome_2))
        )

        child = genome_1[:idx].copy()
        child.extend(genome_2[idx:].copy())
        
        return child
    
    def get_genome_depth(self, genome):
        return len(genome[self.__global_len:])
    
    def get_next_generation(self, fits, pop):
        '''
        create a population of the size defined in the class parameters
        
        parameters:
            fits (array) set of floats taken from each models evaluation performance
            pop (2d array) the current population under evaluation
        '''
        next_generation = []
        for i in range(len(fits)):
#             find the index two fit parents
            p1 = self.select_fittest(fits)
            p2 = self.select_fittest(fits)
        
#             get the genome of each parent
            gn1 = pop[p1]
            gn2 = pop[p2]
            
#             mate the parents
            child = self.crossover(gn1, gn2)
    
#             apply the point mutation to the child
            child = self.point_mutate(child.copy())
#             grow, shrink or retain the depth using the probability applied within the functions
            size_mutate = {
                1: self.grow_mutate(child),
                2: self.shrink_mutate(child),
                3: child}
            p = np.random.randint(1, 4)
            child = size_mutate[p]
            
            next_generation.append(child)
            
        return next_generation
    
ev = Evolution(probability=0.1,highest_is_fittest=False, max_depth=4)

In [ ]:
fits = [0.5,0.3,1,0.6,0.1]
next_gen = ev.get_next_generation(fits, pop)
print(f'Pop size: {[len(g) for g in pop]}')
print(f'Next gen: {[len(g) for g in next_gen]}')

In [ ]:
child = ev.crossover(pop[0].copy(), pop[1].copy())
child
# ev.point_mutate(child)

In [ ]:
pop[0]

In [ ]:
idx = 2
child = []
c1 = pop[0][:idx].copy()
c2 = pop[1][idx:].copy()
print(f'{c1}\n{c2}')

child.extend(c1)
child.extend(c2)
print(child)

In [ ]:
import matplotlib.pyplot as plt 
# fits = [30, 20,10,10000,500,600,700,0.1]
# fits = [0.4,0.3,0.6,0.1,0.7,0.2,0.18,0.1]
fits = [1,2,3,4,5]
# fits = best_results
results = []
pop = pn.get_population(pop_size=20)
for i in range(1000):
    res = ev.select_fittest(fits)
    results.append(fits[res])
    
unique, counts = np.unique(results, return_counts=True)
print(unique, counts)

In [ ]:
idx = ev.select_fittest(best_results)
print(best_results, best_results[idx])

In [ ]:
# pop = pn.get_population(pop_size=5)
pop

In [ ]:
pop_test = pn.get_population(pop_size=10)
c1 = pop_test[0]
c2 = pop_test[1]
print(f'C1{c1}\n\nC2{c2}')

In [ ]:
c0, idx = ev.crossover(c1, c2)
print(f'idx: {idx}\n\nC0{c0}')

In [ ]:

mutated_genome = ev.grow_mutate(c1)
mutated_genome

In [ ]:
mutated_genome = ev.shrink_mutate(c1)
mutated_genome

In [ ]:
genome = c1.copy()
mutated_genome = ev.point_mutate(genome)
mutated_genome

In [ ]:
# use in the model builder to assign a default in the case of a missing parameter
d = {'test':100}
d.get('t', 'default')


# Redundant Code

In [ ]:
# redundant
# class Model_Handler:
    
#     def __init__(self, shape, output_size, global_size, layer_size):
#         self.__shape = shape
#         self.__output_size = output_size
#         # the size of the global parameters eg global_params = params[:global_size]
#         self.__global_size = global_size
#         # the size of each layer's parameters eg n arrays of width layer_size
#         self.__layer_size = layer_size

# #     def build_model(self, global_params, layer_params):
        
# #         # used for the scaling of each filter size
# #         prev_size = global_params[2]
# #         # new empty model
# #         model = keras.Sequential()
# #         # set the input shape
# #         model.add(keras.Input(shape=self.__shape))
# #         # using a reshaped layer parameter array loop each and apply
# #         for size_scale, dropout, rate in layer_params:
# #             layer_size = int(size_scale * prev_size)
# #             # set the layer size as a multiple of the previous using the size_scale param
# #             model.add(keras.layers.Conv2D(filters=layer_size, kernel_size=3, activation="relu"))
# #             model.add(keras.layers.MaxPooling2D(pool_size=2))
# #             # only add a droput layer if param = true
# #             if dropout:
# #                 model.add(keras.layers.Dropout(rate=rate))

# #             prev_size = layer_size

# #         # add the final layers of the model
# #         model.add(keras.layers.Flatten())
# #         model.add(keras.layers.Dense(self.__output_size, activation="softmax"))      

# #         # compile the model
# #         model.compile(optimizer="rmsprop",
# #                       loss="sparse_categorical_crossentropy",
# #                       metrics=["accuracy"])

# #         return model
    
#     def train_model(model, X, y, epochs, split, batch, callbacks):
#         model.fit(
#         dh.X_train[:1000], 
#         dh.y_train[:1000], 
#         epochs=max_epochs, 
#         validation_split=val_split, 
#         batch_size=batch_size, 
#         callbacks=[callbacks]
#     )
        
#     def test_population(self, population):
#         # split into global and layer params
#         global_params = params[:self.__global_size]
#         layer_params = np.array(params[self.__global_size:]).reshape(self.__layer_shape)
        
#         for genome in population:
#             print(genome)
# #             model = self.build_model(global_params, layer_params)

In [ ]:
# redundant
# mh = Model_Handler(
#     shape=(281, 362, 1), 
#     output_size=5, 
#     global_size=population.global_size, 
#     layer_shape=population.layer_shape
# )
# # model = mh.build_model(global_params, layer_params)
# # model.summary()

# mh.test_population(pop)

In [ ]:
# redundant
# # set the params
# max_epochs = 5
# vaidation_split = 0.2
# batch_size = 256

# # create the callbacks
# monitor = 'val_loss'
# checkpoint_path = 'checkpoint_path.keras'

# callbacks = [
#     keras.callbacks.EarlyStopping(
#         monitor=monitor, 
#         patience=3
#     ),
    
# #     keras.callbacks.ModelCheckpoint(
# #         filepath=checkpoint_path, 
# #         monitor=monitor, 
# #         save_best_only=True
# #     )
# ]

# model.fit(
#     dh.X_train[:1000], 
#     dh.y_train[:1000], 
#     epochs=max_epochs, 
#     validation_split=vaidation_split, 
#     batch_size=batch_size, 
#     callbacks=[callbacks]
# )

In [ ]:
# redundant
# initializer = keras.initializers.Ones()
# layer = keras.layers.Conv2D(filters=layer_size, kernel_size=3, activation="relu", kernel_initializer=initializer)